In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from typing import List, Union



# 1. Defina as features (variáveis de entrada) na ordem correta


In [ ]:
# Define as features, na ordem correta
FEATURE_NAMES = [
    'COD_UF_EMIT', 'TIP_FIN_NFE', 'CEST_COMPLETO', 'COD_CST', 'NCM_COMPLETO', 'CFOP_COMPLETO', 'EMIT_CNAE_COMPLETO', 'EMIT_CRT', 
    'EMIT_IND_SN', 'DEST_CNAE_COMPLETO', 'DEST_SITUACAO', 'DEST_IND_SN', 'DEST_POSSUI_IE', 'EMIT_CNAE_DIVISAO', 'EMIT_CNAE_GRUPO', 
    'EMIT_CNAE_CLASSE', 'DEST_CNAE_DIVISAO', 'DEST_CNAE_GRUPO', 'DEST_CNAE_CLASSE', 'NCM_CAPITULO', 'NCM_POSICAO', 'NCM_SUBPOSICAO', 
    'CFOP_NATUREZA', 'CFOP_OPERACAO', 'POSSUI_CEST', 'CEST_SEGMENTO', 'CEST_ITEM']

# Features que devem ser tratadas como Categóricas (para corrigir o erro)
CATEGORICAL_FEATURES = [
    'COD_UF_EMIT', 'TIP_FIN_NFE', 'CEST_COMPLETO', 'COD_CST', 'NCM_COMPLETO', 'CFOP_COMPLETO', 'EMIT_CNAE_COMPLETO', 'EMIT_CRT', 
    'EMIT_IND_SN', 'DEST_CNAE_COMPLETO', 'DEST_SITUACAO', 'DEST_IND_SN', 'DEST_POSSUI_IE', 'EMIT_CNAE_DIVISAO', 'EMIT_CNAE_GRUPO', 
    'EMIT_CNAE_CLASSE', 'DEST_CNAE_DIVISAO', 'DEST_CNAE_GRUPO', 'DEST_CNAE_CLASSE', 'NCM_CAPITULO', 'NCM_POSICAO', 'NCM_SUBPOSICAO', 
    'CFOP_NATUREZA', 'CFOP_OPERACAO', 'POSSUI_CEST', 'CEST_SEGMENTO', 'CEST_ITEM'
]

# 2. Caminho do arquivo do modelo


In [ ]:
MODEL_FILE_PATH = "modelo_lgbm 3.txt"


# 3. Função principal para carregar o modelo e fazer a predição


In [ ]:
def carregar_e_classificar(dados_x: pd.DataFrame) -> Union[np.ndarray, int]:
    """
    Carrega o modelo LightGBM, realiza o pré-processamento necessário
    e faz a classificação dos dados fornecidos.
    """
    try:
        # Carrega o modelo (Booster)
        bst = lgb.Booster(model_file=MODEL_FILE_PATH)
    except lgb.basic.LightGBMError as e:
        print(f"Erro ao carregar o modelo: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

    # Garante que as colunas do DataFrame estão na ordem correta
    dados_para_predicao = dados_x[FEATURE_NAMES].copy()

    # **ETAPA DE CORREÇÃO:** Converte as colunas categóricas para o tipo 'category'
    for col in CATEGORICAL_FEATURES:
        if col in dados_para_predicao.columns:
            # Converte para string antes de category para tratar valores mistos (num/str)
            dados_para_predicao[col] = dados_para_predicao[col].astype(str).astype('category')
    
    # Realiza a predição. O LightGBM agora deve reconhecer as colunas categóricas.
    probabilidades = bst.predict(dados_para_predicao)

    # A classificação final é a classe com a maior probabilidade.
    classificacao_y = np.argmax(probabilidades, axis=1)

    # Retorna o valor ou a lista de valores
    if len(classificacao_y) == 1:
        return classificacao_y[0]
    return classificacao_y

# 4. Exemplo de Uso


In [ ]:
import pandas as pd
from io import StringIO

# A string de dados fornecida, formatada para simular a entrada de um arquivo tabulado
dados_string = """
SEQ_NFE\tCOD_UF_EMIT\tTIP_FIN_NFE\tID_PROD\tCEST_COMPLETO\tCOD_CST\tNCM_COMPLETO\tCFOP_COMPLETO\tEMIT_CNAE_COMPLETO\tEMIT_CRT\tEMIT_IND_SN\tDEST_CNAE_COMPLETO\tDEST_SITUACAO\tTIPO_IMPOSTO\tDEST_IND_SN\tDEST_POSSUI_IE\tEMIT_CNAE_DIVISAO\tEMIT_CNAE_GRUPO\tEMIT_CNAE_CLASSE\tDEST_CNAE_DIVISAO\tDEST_CNAE_GRUPO\tDEST_CNAE_CLASSE\tNCM_CAPITULO\tNCM_POSICAO\tNCM_SUBPOSICAO\tCFOP_NATUREZA\tCFOP_OPERACAO\tPOSSUI_CEST\tCEST_SEGMENTO\tCEST_ITEM
435959728\tSP\t1\t2928998889\t0\t0\t96082000\t6102\tAUSENTE\t1\tS\t55108010\t2\tSTDIF\t0\t1\tAUSENTE\tAUSENTE\tAUSENTE\t55\t551\t5510\t96\t9608\t960820\t6\t102\t0\tAUSENTE\tAUSENTE
435959728\tSP\t1\t2928998890\t0\t0\t44191900\t6101\tAUSENTE\t1\tS\t55108010\t2\tANT\t0\t1\tAUSENTE\tAUSENTE\tAUSENTE\t55\t551\t5510\t44\t4419\t441919\t6\t101\t0\tAUSENTE\tAUSENTE
"""

# Usando StringIO para tratar a string como um arquivo
# O parâmetro 'sep='\t'' indica que a tabulação é o separador
df = pd.read_csv(StringIO(dados_string), sep='\t')

print("DataFrame 'df' criado com sucesso:")
print("-" * 50)
print(df.head())
print("-" * 50)
print(f"Dimensão do DataFrame: {df.shape}")
print(f"Colunas (Total {len(df.columns)}): {df.columns.tolist()}")


# Realiza a classificação
resultado_y = carregar_e_classificar(df)

if resultado_y is not None:
    print("--- Resultado da Classificação (Y) ---")
    if isinstance(resultado_y, int):
            print(f"Classificação prevista: {resultado_y}")
    else:
        df['CLASSIFICACAO_Y'] = resultado_y
        print(df[['CLASSIFICACAO_Y']])
        print(f"\nResultado completo (array): {resultado_y}")

In [12]:
import pandas as pd
import lightgbm as lgb
from io import StringIO
import numpy as np

# 1. Definição das Features e Identificação dos Tipos
# A lista de feature_names foi extraída do arquivo 'modelo_lgbm 3.txt'
FEATURE_NAMES = [
    'COD_UF_EMIT', 'TIP_FIN_NFE', 'CEST_COMPLETO', 'COD_CST', 'NCM_COMPLETO', 
    'CFOP_COMPLETO', 'EMIT_CNAE_COMPLETO', 'EMIT_CRT', 'EMIT_IND_SN', 'DEST_CNAE_COMPLETO', 
    'DEST_SITUACAO', 'DEST_IND_SN', 'DEST_POSSUI_IE', 'EMIT_CNAE_DIVISAO', 'EMIT_CNAE_GRUPO', 
    'EMIT_CNAE_CLASSE', 'DEST_CNAE_DIVISAO', 'DEST_CNAE_GRUPO', 'DEST_CNAE_CLASSE', 'NCM_CAPITULO', 
    'NCM_POSICAO', 'NCM_SUBPOSICAO', 'CFOP_NATUREZA', 'CFOP_OPERACAO', 'POSSUI_CEST', 
    'CEST_SEGMENTO', 'CEST_ITEM'
]

# Pela natureza dos dados (códigos, divisões, indicadores 'S/N', etc.), 
# a prática comum com LightGBM é tratar todas essas colunas como categóricas, 
# exceto por chaves numéricas que não são usadas na predição (SEQ_NFE, ID_PROD).
CATEGORICAL_FEATURES = FEATURE_NAMES.copy()

# 2. Criação do DataFrame de Teste
# Os dados de teste são baseados nos valores que você forneceu, removendo SEQ_NFE, ID_PROD e TIPO_IMPOSTO.
data = {
    'COD_UF_EMIT': ['SP', 'SP'],
    'TIP_FIN_NFE': [1, 1],
    'CEST_COMPLETO': [0, 0],
    'COD_CST': [0, 0],
    'NCM_COMPLETO': [96082000, 44191900],
    'CFOP_COMPLETO': [6102, 6101],
    'EMIT_CNAE_COMPLETO': ['AUSENTE', 'AUSENTE'],
    'EMIT_CRT': [1, 1],
    'EMIT_IND_SN': ['S', 'S'],
    'DEST_CNAE_COMPLETO': [55108010, 55108010],
    'DEST_SITUACAO': [2, 2],
    'DEST_IND_SN': [0, 0],
    'DEST_POSSUI_IE': [1, 1],
    'EMIT_CNAE_DIVISAO': ['AUSENTE', 'AUSENTE'],
    'EMIT_CNAE_GRUPO': ['AUSENTE', 'AUSENTE'],
    'EMIT_CNAE_CLASSE': ['AUSENTE', 'AUSENTE'],
    'DEST_CNAE_DIVISAO': [55, 55],
    'DEST_CNAE_GRUPO': [551, 551],
    'DEST_CNAE_CLASSE': [5510, 5510],
    'NCM_CAPITULO': [96, 44],
    'NCM_POSICAO': [9608, 4419],
    'NCM_SUBPOSICAO': [960820, 441919],
    'CFOP_NATUREZA': [6, 6],
    'CFOP_OPERACAO': [102, 101],
    'POSSUI_CEST': [0, 0],
    'CEST_SEGMENTO': ['AUSENTE', 'AUSENTE'],
    'CEST_ITEM': ['AUSENTE', 'AUSENTE']
}

# A string de dados fornecida, usando ponto e vírgula (;) como separador
dados_string = """
SEQ_NFE;COD_UF_EMIT;TIP_FIN_NFE;ID_PROD;CEST_COMPLETO;COD_CST;NCM_COMPLETO;CFOP_COMPLETO;EMIT_CNAE_COMPLETO;EMIT_CRT;EMIT_IND_SN;DEST_CNAE_COMPLETO;DEST_SITUACAO;TIPO_IMPOSTO;DEST_IND_SN;DEST_POSSUI_IE;EMIT_CNAE_DIVISAO;EMIT_CNAE_GRUPO;EMIT_CNAE_CLASSE;DEST_CNAE_DIVISAO;DEST_CNAE_GRUPO;DEST_CNAE_CLASSE;NCM_CAPITULO;NCM_POSICAO;NCM_SUBPOSICAO;CFOP_NATUREZA;CFOP_OPERACAO;POSSUI_CEST;CEST_SEGMENTO;CEST_ITEM
436983724;PR;1;2934276265;0;0;44201900;6101;AUSENTE;1;S;AUSENTE;0;ANT;0;0;AUSENTE;AUSENTE;AUSENTE;AUSENTE;AUSENTE;AUSENTE;44;4420;442019;6;101;0;AUSENTE;AUSENTE
437027824;PB;1;2934465641;0;10;19041000;6401;10643000;3;N;47121000;2;ST;0;1;10;106;1064;47;471;4712;19;1904;190410;6;401;0;AUSENTE;AUSENTE
"""

# Usando StringIO para ler a string como um arquivo
# O parâmetro 'sep=';'' garante a leitura correta dos dados
df_test = pd.read_csv(StringIO(dados_string), sep=';')

# Criamos o DataFrame de teste
# df_test = pd.DataFrame(data, columns=FEATURE_NAMES)

# 3. Conversão de Tipos (Essencial para LightGBM)
# Convertemos todas as colunas identificadas como categóricas para o dtype 'category'
for col in CATEGORICAL_FEATURES:
    # Garantir que valores como 'AUSENTE' sejam tratados corretamente
    df_test[col] = df_test[col].replace('AUSENTE', np.nan) 
    df_test[col] = df_test[col].astype('category')

# 4. Simulação de Carga e Predição do Modelo (Placeholder)
# O modelo real estaria contido no arquivo 'modelo_lgbm 3.txt'.
# Para o teste, apenas mostramos como seria a preparação e a chamada de predição.

try:
    # Simulação de carregamento do modelo LightGBM (Se o arquivo fosse binário)
    bst = lgb.Booster(model_file='modelo_lgbm 3.txt')
    predictions = bst.predict(df_test, raw_score=False)
    
    # argmax retorna o índice (0 a 4) da classe com maior probabilidade
    classificacao_y_indices = np.argmax(predictions, axis=1)


    print(classificacao_y_indices)

except Exception as e:
    # Se o modelo fosse carregado, ele falharia porque o arquivo é texto, não binário.
    # Esta exceção é apenas para demonstrar o fluxo ideal.
    print(f"\nErro (Esperado): Não foi possível carregar o modelo LightGBM a partir do arquivo de texto.\nDetalhes: {e}")


Erro (Esperado): Não foi possível carregar o modelo LightGBM a partir do arquivo de texto.
Detalhes: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: TIPO_IMPOSTO: object


C:\Users\ajsantos\AppData\Local\Temp\ipykernel_28776\2667272179.py:72: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test[col] = df_test[col].replace('AUSENTE', np.nan)
